In [3]:
# ===============================
# Livestock Emissions ML Analysis
# ===============================

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
import shap

# -------------------------------
# 1. Load the data
# -------------------------------
livestock_df = pd.read_csv("GLEAM_LivestockEmissions.csv")

# Inspect data
print(livestock_df.head())
print(livestock_df.info())

ModuleNotFoundError: No module named 'shap'

In [ ]:
# -------------------------------
# 2. Basic preprocessing
# -------------------------------

# Fill missing values with 0 (or consider more sophisticated imputation)
livestock_df.fillna(0, inplace=True)

# Features and target for regression
numeric_features = [
    "Production (kg protein)",
    "Total CO2 emissions (kg CO2e)",
    "Total CH4 emissions (kg CO2e)",
    "Total N2O emissions (kg CO2e)",
    "Feed, CO2 (kg CO2e)",
    "Feed, CH4 (kg CO2e)",
    "Feed: fertilizer & crop residues, N2O (kg CO2e)",
    "Feed: applied & deposited manure, N2O (kg CO2e)",
    "LUC: soy & palm, CO2 (kg CO2e)",
    "LUC: pasture expansion, CO2 (kg CO2e)",
    "Enteric fermentation, CH4 (kg CO2e)",
    "Manure management, CH4 (kg CO2e)",
    "Manure management, N2O (kg CO2e)",
    "Direct energy, CO2 (kg CO2e)",
    "Indirect energy, CO2 (kg CO2e)",
    "Postfarm, CO2 (kg CO2e)"
]

categorical_features = ["Region", "Animal species", "Production system", "Commodity"]

target = "Emission Intensity (kg CO2e per kg protein)"

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(), categorical_features)
    ]
)

In [ ]:
# -------------------------------
# 3. Regression Model
# -------------------------------
X = livestock_df[numeric_features + categorical_features]
y = livestock_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=200, random_state=42))
])

rf_pipeline.fit(X_train, y_train)

y_pred = rf_pipeline.predict(X_test)
print("R² score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
# -------------------------------
# 4. Feature Importance with SHAP
# -------------------------------
# Extract preprocessed numeric data for SHAP
X_train_transformed = preprocessor.fit_transform(X_train)
regressor = rf_pipeline.named_steps["regressor"]

explainer = shap.Explainer(regressor, X_train_transformed)
shap_values = explainer(X_train_transformed)

shap.plots.bar(shap_values, max_display=15)

In [ ]:
# -------------------------------
# 5. Clustering (Emission Profiles)
# -------------------------------
cluster_features = ["Total CO2 emissions (kg CO2e)", "Total CH4 emissions (kg CO2e)", "Total N2O emissions (kg CO2e)"]
X_cluster = StandardScaler().fit_transform(livestock_df[cluster_features])

kmeans = KMeans(n_clusters=3, random_state=42)
livestock_df["Cluster"] = kmeans.fit_predict(X_cluster)

# Cluster visualization
sns.scatterplot(
    data=livestock_df,
    x="Total CH4 emissions (kg CO2e)",
    y="Total CO2 emissions (kg CO2e)",
    hue="Cluster",
    style="Production system",
    palette="Set2",
    s=100
)
plt.title("Livestock Emission Clusters")
plt.show()


In [ ]:
# -------------------------------
# 6. Regression Trend Plot Example
# -------------------------------
sns.regplot(
    data=livestock_df,
    x="Production (kg protein)",
    y="Total GHG emissions (kg CO2e)",
    logx=True,
    scatter_kws={'s':50, 'alpha':0.6}
)
plt.title("Production vs Total GHG Emissions")
plt.xlabel("Production (kg protein, log scale)")
plt.ylabel("Total GHG emissions (kg CO2e)")
plt.show()